## logging 實戰

> 歷程記錄

log 都是和 try exception 搭配使用

把 logging 當成高級的 print 只是他是用來記錄資料流通的足跡!

In [2]:
from typing import List, Callable, Dict, Set

### 1. 設定 logging 的基礎配置
在應用程式的入口文件（例如 __main__ 模組）中設定 logging 的基本配置，這樣可以確保所有模組的日誌輸出格式和等級一致。可以通過 logging.basicConfig 來設定：

在 Python 中，你可以使用 logging.basicConfig() 來初始化日誌系統的基本配置。這個方法允許你設定全局的日誌等級、格式、輸出目標（如文件或控制台）等。這種方式適合於簡單的應用場景，當你不需要非常複雜的日誌配置時，可以使用 basicConfig 來快速設置。

logging.basicConfig() 只能被調用一次，如果你在程式中多次調用它，只有第一次調用會生效。如果需要更靈活的配置，應該使用自定義的 logger 和 handler。

In [18]:
import logging

logging.basicConfig(
    level=logging.INFO, # 基礎 level
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler("app.log"),  # 日誌輸出到檔案
        logging.StreamHandler()          # 日誌輸出到控制台
    ]
)


### 2. 在每個模組或類中創建 logger
為了方便管理和追蹤日誌，一般在每個模組或類中創建一個 logger 實例，這樣可以使日誌更加細化。

In [19]:
class Step:
    def __init__(self, name: str, func: Callable):
        self.name = name
        self.func = func
        self.logger = logging.getLogger(__name__)  # 創建 logger

    def execute(self, *args, **kwargs):
        self.logger.info(f"Executing step: {self.name}")
        try:
            result = self.func(*args, **kwargs)
            self.logger.info(f"Step {self.name} completed successfully with result: {result}")
            return result
        except Exception as e:
            self.logger.error(f"Error occurred in step {self.name}: {str(e)}")
            raise


In [20]:
def add_number(a:int, b:int):
    sum = a + b

    return sum


step_add_number = Step("add_number", add_number)
step_add_number.execute(10, 100)


2024-08-20 23:48:03,220 - __main__ - INFO - Executing step: add_number
2024-08-20 23:48:03,220 - __main__ - INFO - Step add_number completed successfully with result: 110


110

In [21]:
def multiply_number(a:int, b:int, c:int):
    return a * b * c

step_multiply_number = Step("multiply_number", multiply_number)
step_multiply_number.execute(10, 100, 200)



2024-08-20 23:48:03,229 - __main__ - INFO - Executing step: multiply_number
2024-08-20 23:48:03,229 - __main__ - INFO - Step multiply_number completed successfully with result: 200000


200000

### 3. 日誌等級的選擇
使用適當的日誌等級來表示訊息的嚴重性：

- DEBUG: 詳細的開發和調試信息。
- INFO: 一般運行信息，如開始執行某步驟。
- WARNING: 潛在的問題，但不影響程序的正常運行。
- ERROR: 嚴重問題，程序可能無法正常運行。
- CRITICAL: 致命問題，程序必須停止。


#### 設計日誌等級策略
根據開發、測試、運行和調試。以下是常見的設計技巧：

- 開發環境: 設定較低的日誌等級（如 DEBUG），這樣可以捕捉到所有詳盡的開發和調試信息。
- 測試環境: 設定為 INFO 或 WARNING，重點捕捉系統運行狀況和潛在問題。
- 生產環境: 設定為 ERROR 或更高，這樣可以避免過多不必要的日誌輸出，僅捕捉真正影響系統運行的問題。

#### > 使用 DEBUG 等級記錄詳盡的開發和調試信息

In [22]:
logger = logging.getLogger(__name__)  # 創建 logger, 紀錄當前模組的狀態

def complex_calculation(x, y):
    logger.debug(f"Starting complex_calculation with x={x}, y={y}")
    result = x * y  
    logger.debug(f"Result of complex_calculation: {result}")
    return result


#### > 使用 INFO 等級記錄一般運行信息

In [23]:
logger = logging.getLogger(__name__)  # 創建 logger, 紀錄當前模組的狀態

def start_process():
    logger.info("Starting the main process")
    # 處理邏輯
    logger.info("Main process completed successfully")


#### > 使用 WARNING 等級記錄潛在問題

In [24]:
logger = logging.getLogger(__name__)  # 創建 logger, 紀錄當前模組的狀態

def fetch_data():
    data = external_service_call()
    if data is None:
        logger.warning("Received no data from the external service")
    return data


#### > 使用 ERROR 等級記錄嚴重問題

In [25]:
logger = logging.getLogger(__name__)  # 創建 logger, 紀錄當前模組的狀態

def process_data(data):
    try:
        result = perform_complex_operation(data)
    except Exception as e:
        logger.error(f"Error processing data: {str(e)}")
        raise


#### > 使用 CRITICAL 等級記錄致命問題

In [26]:
logger = logging.getLogger(__name__)  # 創建 logger, 紀錄當前模組的狀態

def critical_failure():
    if critical_condition_met():
        logger.critical("Critical failure detected! Shutting down the system.")
        shutdown_system()


日誌等級應該與系統通知作整合
在設計中，ERROR 和 CRITICAL 等級的日誌應該與通知系統集成，例如通過電子郵件、簡訊或其他即時通知方式告知相關人員，以便立即採取行動。

## 日誌過濾

In [42]:
# 在大型應用中，不同模組或套件可能需要不同的日誌配置。可以在各自的模組中設置 logger 並指定特定的配置：
# logger = logging.getLogger('module.submodule')
logger = logging.getLogger(__name__)  # 創建 logger, 紀錄當前模組的狀態
logger.setLevel(logging.WARNING)

def sum(a, b):

    sum = a + b
    logger.info(f"sum is {sum}")
    return sum

total = sum(10, 4)

print(total)


14


### 4. 使用不同的日誌處理器
日誌可以同時輸出到多個地方，例如控制台、文件、或遠端日誌伺服器。使用 logging 模組中的不同處理器 (handlers) 可以實現這一點，例如：


現在系統有兩種需求
- console 的輸出層級 INFO
- file 的輸出 DEBUG


注意!!!
logging.basicConfig() 導入時機點

1. 為什麼通常在前期導入？
早期記錄: 如果在程序運行過程中發生了錯誤或需要調試的信息，早期配置日誌系統可以確保這些信息被正確記錄。例如，在程序啟動、加載配置、初始化關鍵模塊等早期階段發生的事件。
全局影響: logging.basicConfig() 配置影響全局的日誌設置，早期設置有助於確保全局日誌設置的統一性。這對於大多數應用程序來說是期望的行為。

2. 晚期導入的情況
動態配置: 在某些場景下，你可能會根據運行時的參數或條件來動態配置日誌系統。這時候，可以在需要時調用 logging.basicConfig() 來設置或重設日誌配置。例如，根據命令行參數來決定日誌輸出的詳細程度或輸出位置。
條件性配置: 如果你有多個模組或程式碼段需要根據某些條件來配置日誌，你可以在相應的地方配置而不是統一在初始化階段。

3. 需要注意的部分
第一次調用影響最大: logging.basicConfig() 在程序中第一次調用時會生效，後續再調用將不會有影響（除非移除所有 handler 或重新加載 logging 模組）。所以如果你在後期才導入，可能會錯過記錄一些早期的重要信息。
影響範圍: 如果你在程序的中間階段或後期調用 logging.basicConfig()，要小心已經存在的日誌配置可能已經被其他模組或代碼設置了，這時調用可能無法覆蓋這些配置。


In [69]:
import logging
import importlib

# 重新加載 logging 模組 可以清除之前的logger設定
importlib.reload(logging)


<module 'logging' from 'c:\\Users\\xdxd2\\AppData\\Local\\Programs\\Python\\Python311\\Lib\\logging\\__init__.py'>

## 使用不同的日誌處理器 版本A - 使用 logging.basicConfig 前期做預設

缺點: 遇到根據不同平台輸出不同紀錄的要求，無法達成!

In [70]:
import logging

# 使用 basicConfig 設定日誌配置
logging.basicConfig(
    level=logging.INFO,  # 設定全局 logger 的最低日誌級別
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',  # 設定日誌格式
    datefmt='%Y-%m-%d %H:%M:%S',  # 設定日期時間格式
    handlers=[
        logging.StreamHandler(),  # Console Handler
        logging.FileHandler('app.log', mode='w')  # File Handler
    ]
)

# 創建 logger
logger = logging.getLogger(__name__)

# 測試日誌輸出
logger.debug('This is a debug message')    # 只會記錄在文件中
logger.info('This is an info message')     # 會顯示在控制台和文件中
logger.warning('This is a warning message')# 會顯示在控制台和文件中
logger.error('This is an error message')   # 會顯示在控制台和文件中
logger.critical('This is a critical message') # 會顯示在控制台和文件中


2024-08-21 01:07:43 - __main__ - INFO - This is an info message
2024-08-21 01:07:43 - __main__ - WARNING - This is a warning message
2024-08-21 01:07:43 - __main__ - ERROR - This is an error message
2024-08-21 01:07:43 - __main__ - CRITICAL - This is a critical message


## 使用不同的日誌處理器 版本B - handler 客製化
使用 logger = logging.getLogger(__name__) 建立初始化的 logging 物件再透過 handler 進行不同行為的客製化 *(推薦!! 比較靈活)

handler 的意思是小助手，所以遇到各種要求就必須聘請不同小助手幫助紀錄

In [67]:
import logging
import importlib

# 重新加載 logging 模組 可以清除之前的logger設定
importlib.reload(logging)


<module 'logging' from 'c:\\Users\\xdxd2\\AppData\\Local\\Programs\\Python\\Python311\\Lib\\logging\\__init__.py'>

In [68]:

# 創建 logger
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)  # 設定 logger 的最低日誌等級

# 創建 Console Handler 並設定日誌等級
console_handler = logging.StreamHandler()
console_handler.setLevel(logging.INFO)

# 創建 File Handler 並設定日誌等級
file_handler = logging.FileHandler('app.log')
file_handler.setLevel(logging.DEBUG)

# 創建 Formatter
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')

# 為 handler 設置 formatter
console_handler.setFormatter(formatter)
file_handler.setFormatter(formatter)

# 為 logger 添加 handler
logger.addHandler(console_handler)
logger.addHandler(file_handler)

# 測試日誌輸出
logger.debug('This is a debug message')    # 只會記錄在文件中，因為 console 的等級是 INFO
logger.info('This is an info message')     # 會顯示在控制台和文件中
logger.warning('This is a warning message')# 會顯示在控制台和文件中
logger.error('This is an error message')   # 會顯示在控制台和文件中
logger.critical('This is a critical message') # 會顯示在控制台和文件中


2024-08-21 01:06:39,220 - __main__ - INFO - This is an info message
2024-08-21 01:06:39,220 - __main__ - WARNING - This is a warning message
2024-08-21 01:06:39,221 - __main__ - ERROR - This is an error message
2024-08-21 01:06:39,221 - __main__ - CRITICAL - This is a critical message


# 日誌輪轉 - 指定保留天數

In [1]:
import logging
import importlib

# 重新加載 logging 模組 可以清除之前的logger設定
importlib.reload(logging)

<module 'logging' from 'c:\\Users\\xdxd2\\AppData\\Local\\Programs\\Python\\Python311\\Lib\\logging\\__init__.py'>

In [2]:
import logging
from logging.handlers import RotatingFileHandler

# 設置 logger
logger = logging.getLogger("test")
logger.setLevel(logging.DEBUG)  # 設置最低日誌級別

# 創建 RotatingFileHandler
rotating_handler = RotatingFileHandler(
    'app.log',      # 日誌文件名稱
    maxBytes=2000,  # 單個文件的最大大小（以字節為單位）
    backupCount=2   # 最多保留的備份文件數量
)

# 設置格式化器
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
rotating_handler.setFormatter(formatter)

# 將處理器添加到 logger
logger.addHandler(rotating_handler)

# 測試輸出多條日誌，確保文件達到 maxBytes 並觸發輪轉
for i in range(100):
    logger.debug(f'This is debug message {i}')

logger.info('Logging to a rotating file with backup count of 5.')
logger.warning('This is a warning that should be logged in the rotating file.')
logger.error('This is an error message.')

# 觀察目錄下的文件輪轉情況
